## Backtesting Trading Strategy in Python

In [123]:
import pandas as pd
import datetime
import talib
import numpy as np
import nifty100list
watchlist = nifty100list.nifty100
# watchlist

In [124]:
def dataframe_initial_modified(df):
    # entry mai closing price pe le rha isiliye stoploss bhi 
    # df["long_stoploss"] = round(df["close"] - (df["close"]*stoplossPercent)/100, 2) #galat hai ye 
    # df["short_stoploss"] = round(df["close"]  +(df["close"]*stoplossPercent)/100, 2) #galat hai ye
    df["today_change"] = round(df["close"] - df["open"], 2)
    df["prev_close"] = df["close"].shift(1)
    
    return df

In [125]:
# common function for backtesting

# function to get the dataframe from csv 
def readDataFile(path):
    df = pd.read_csv(path, parse_dates=True, index_col=["date"])
    df["time"] = df.index.strftime("%H%M")
    return df

# function to make column of indicator in the dataFrame
def pushIndicator(df):
    
    df['vol_fast'] =  round(talib.MA(df['volume'], timeperiod=10),1)
    df['vol_slow'] =  round(talib.EMA(df['volume'], timeperiod=30),1)
    
    df["sma_fast"] = round(talib.SMA(df['close'], timeperiod=10),1)
    df["sma_slow"] = round(talib.SMA(df['close'], timeperiod=50),1)
    
    
    df['ema_fast'] =  round(talib.EMA(df['close'], timeperiod=10),1)
    df['ema_slow'] =  round(talib.EMA(df['close'], timeperiod=30),1)
    
    df['rsi'] = round(talib.RSI(df['close'], timeperiod=10), 1)
    return df[29:]

In [126]:
def long_position(df):
    # 1-> long_entry  0-> no postion(exit if had prev)
    ###############  Long Entry , Exit, position, continue position
    df["long_entry"] = df.apply(longEntry, axis=1) 
    df["long_exit"] = df.apply(longExit, axis=1) 
    # 
    df["long_position"]= np.nan
    # jaha jaha long entry == true, position = 1 && long exit == true position = 0
    df.loc[df.long_entry, "long_position"] = 1
    df.loc[df.long_exit, "long_position"] = 0
    # position maintain kro until position change nn ho jaye
    df.long_position = df.long_position.fillna(method = "ffill")
    # print(df)
    return df    

def short_position(df):
    # -1-> short_entry  0-> no postion(exit if had prev)
    ###############  short Entry , Exit, position, continue position 
    df["short_entry"] = df.apply(shortEntry, axis=1) 
    df["short_exit"] = df.apply(shortExit, axis=1) 
    # crete column
    df["short_position"]= np.nan
    # jaha jaha short entry == true, position = -1 &&  long exit == true, position = 0
    df.loc[df.short_entry, "short_position"] = -1
    df.loc[df.short_exit, "short_position"] = 0
    # position maintain kro until position change nn ho jaye
    df.short_position = df.short_position.fillna(method = "ffill")
    return df

In [127]:
def backtest_lafra(df):

    # prev_position is needed in num_trade 
    df["prev_position"] = df["position"].shift(1)
    df["prev_position"] = df["prev_position"].fillna(0)
    
    # pnl column
    df.loc[df["position"] != df["position"].shift(2), "pnl"] = round((df["close"] - df["open"])*quantity* df["position"].shift(1), 2)
    df.loc[df["position"] == df["position"].shift(2), "pnl"] = round((df["close"] - df["close"].shift(1))*quantity* df["position"].shift(1), 2)
    

    
    
    # cumulative  pnl
    df["cumpnl"] = round(df.pnl.cumsum(), 2)
    
    # ye dhayan de ki num_trade long or short ko kaise handel krega -> soln1 = dono ek sath mt chala 
    # Number of trade happened including buy and sell
    df.loc[df["position"] != df["prev_position"], "num_trade"] = 1
    df["num_trade"] = df.num_trade.shift(1)

    # step 1 + step 2 => profit per trade => was it prfotable or it was loss. 
    # step 1 -> use [cumulative_pnl] to get idea how much i made in the last trade?
    df.loc[((df["num_trade"]==1 ) & (df["prev_position"]==0)), "num_wins_helper"] = df["cumpnl"]
    df.num_wins_helper = df.num_wins_helper.fillna(method = "ffill")
    df.num_wins_helper =  df.num_wins_helper.fillna(0)
    #step 2 -> #num of profitable days or no. of lossing day
    df.loc[df["num_wins_helper"] > df["num_wins_helper"].shift(1), "num_wins"] = 1
    
    # pnl with stoploss
    df.loc[((df["num_trade"]==1 ) & (df["prev_position"]==0)), "pnl_a"] = df["num_wins_helper"] - df["num_wins_helper"].shift(1) 
    df.loc[df["pnl_a"]<-stoploss, "pnl_a"] = -stoploss 
    
    # df["long_stoploss"] = round(df["close"] - (df["close"]*stoplossPercent)/100, 2) #galat hai ye 
    # df["short_stoploss"] = round(df["close"]  +(df["close"]*stoplossPercent)/100, 2) #galat hai ye
    

    
    return df 

In [128]:
def backtest_result(df):

    # withouot stoploss
    # pnl = df[-1:]["cumpnl"][0]
    # pnl_percent =  round(df[-1:]["cumpnl"][0]/per_trade_capital*100, 2)
    
    
    # with_stoploss
    pnl = sum(df.pnl_a.fillna(0))
    pnl_percent =  round(pnl/per_trade_capital*100, 2)
    
    
    total_trade =  df["num_trade"].sum()
    num_signals = total_trade/2
    
    num_wins = df["num_wins"].sum()
    num_loss = total_trade/2 - num_wins
    
    hit_ratio = round(num_wins/num_signals*100, 0)
    period_return = round((df.iloc[-1]["close"] - df.iloc[0]["open"])/df.iloc[0]["open"]*100 , 2)
    
    ### result of backtest  
    print("Backtest P&L                 ", pnl)
    print("Backtest P&L %               ", pnl_percent)
    print("Total number of signals      ", num_signals)  
    print("Number of wins               ", num_wins)
    print("Number of losses             ",num_loss )
    print("win_percent                  ", num_wins/num_signals*100) 
    # print("Winning streak ", 4)
    # print("Max gains ", df["pnl"].max())
    # print("Max loss ", df["pnl"].min())
    # print("Avg gain/winning trade ", 1000.0000)
    # print("Avg loss/losing trade ", 0.0000)
    # print("Max Drawdown ", 0.3653)
    print("Period High                  ", df["high"].max())
    print("Period Low                   ", df["low"].min())
    print("Period return %              ", period_return)
    print("capital Deployed             ", per_trade_capital)
    print("Quantity Deployed            ", quantity)
    print("\n")
    
    
    
    # filter best results to & search in result database for more info
    # print important metrices 
    if(pnl_percent > period_return):
        print(ticker, "         excesss =  ",round(pnl_percent - period_return, 2), "       pnl% " , pnl_percent, "     hit_ratio", hit_ratio)
    return df

    # **isse bhi implement karo** 
    # print("slippage + dp charge + brokerage charge ")
    # transcation details implement karo

In [129]:
# momentum + strength of momentum(RSI) + trend(MA)
def longEntry(df):
    if ():
        return True
    else:
        return False
        
def longExit(df):
    if ():
        return True
    else:
        return False
    
def shortEntry(df):
    if ():
        return True
    else:
        return False
        
def shortExit(df):   
    if ():
        return True
    else:
        return False

In [130]:
def longEntry(df):
    # if (df["sma_fast"]> df["sma_slow"]):
    #     return True
    if (df["ema_fast"]> df["ema_slow"]) and (df["rsi"] > 55):
        return True
    else:
        return False
        
def longExit(df):
    # if (df["ema_fast"] < df["ema_slow"]):
    #      return True
    if(int(df["time"]) >= 1505):
        return True
    # if (df["rsi"] <40):
    #     return True
    else:
        return False
    
def shortEntry(df):
    if (df["vol_fast"] < df["vol_slow"]) and  (df["ema_fast"] < df["ema_slow"]) and (df["rsi"] < 30):
        return True
    else:
        return False
        
def shortExit(df):   
    if (df["vol_fast"] > df["vol_slow"]) and  (df["ema_fast"] > df["ema_slow"]) and (df["rsi"] > 30):
        return True
    else:
        return False

In [131]:
# for(int i=0; i< no. of stock; i++){???}  => for each stocks in the baskets
for ticker in watchlist:
    time = "min15"
    df = readDataFile("data/" +time + "/"+ ticker + ".csv")

    df = pushIndicator(df)    
    
    # per stock metrices
    capital = 100000
    per_trade_capital = capital / 10  # Rs. 10,000
    quantity =  round(per_trade_capital / df["close"][-1], 0)
    stoplossPercent = 1    
    stoploss = per_trade_capital * stoplossPercent /100
    

    df = dataframe_initial_modified(df)


    df = long_position(df)
    df = short_position(df)
   
    # overall position ==  long + short  ==  mast cheez hai
    #df["num_trade"] me check kr problem to nhi aaa rha long + short dono ek sath le lene se????? =>soln-> df["short"] = 0 
    df["short_position"] = 0
    # df["long_position"] = 0
    
    df["position"] = df.long_position + df.short_position   
    df["position"] = df["position"].fillna(0)

    df = backtest_lafra(df)
    df = backtest_result(df)
    
    # graph(df)
    
    break
    # df.to_csv("result/" + time + "/"+ ticker + ".csv")


df.to_csv("abc.csv")

Backtest P&L                  892.25
Backtest P&L %                8.92
Total number of signals       88.5
Number of wins                39.0
Number of losses              49.5
win_percent                   44.06779661016949
Period High                   2589.0
Period Low                    1900.0
Period return %               -6.36
capital Deployed              10000.0
Quantity Deployed             5.0


ACC-EQ          excesss =   15.28        pnl%  8.92      hit_ratio 44.0
